Import data and create datframes:
We will be working with two datasets, one from Foursquare and one from the West Virginia Department of Transportation.

#### Dataset description
The Foursquare dataset corresponds to all businesses in a 10-mile radious of the Falling Water's WV town. From this dataset we are looking to learn the number and type of businesses in the area, their popularity, and customer's population, as well as the business proximity to potential locations. From the dataset we will also learn if the identified business are competitors or business that could attract potential fast food customers.

The second dataset contains traffic survey for the areas, specifically, traffic volume by the hours for highway exits near potential locations. We hope to learn the traffic patterns and volume to consider potential customers volume other than local residents.

The following section pulls the datasets and create the corresponding dataframes

In [6]:
# Install necessary libraries 
!conda install -c conda-forge folium=0.5.0 --yes
print('Folium installed')
!conda install -c conda-forge geopy --yes
print('geopy library install')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [20]:
# Import necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # Map plotting library
print('Libraries imported')

Libraries imported


In [21]:
#Set up Foursquare credentials
CLIENT_ID = '3DAUBXTARLYYR4GT30CHA5Z5WZQZMUXL54WPZFFLRTFK34CH' # your Foursquare ID
CLIENT_SECRET = 'FC2KX2BX4WZHA4XZDRQ14HKORJNGDGLYWJ0NKC20R2VKXNKP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3DAUBXTARLYYR4GT30CHA5Z5WZQZMUXL54WPZFFLRTFK34CH
CLIENT_SECRET:FC2KX2BX4WZHA4XZDRQ14HKORJNGDGLYWJ0NKC20R2VKXNKP


Set up variables to be used in the Foursquare search request URL

In [22]:
address = 'Falling Waters, West Virginia'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Falling Waters, WV latitude is {}, and the longitude is {}".format(latitude, longitude))

Falling Waters, WV latitude is 39.5592639, and the longitude is -77.8908301


The following section sets up the Foursquare URL to search for all business locations in a 10-miles radius. Something to notice here is that we are not searching by search_queary which queries the name of the business, but by category. For this, I had to include the variable &category to the URL and populate the category value to the category for "Fast Food" as found in the Foursquare website.

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&category=`**CATEGORY**`&radius=`**RADIUS**`&limit=`**LIMIT**


In [23]:
search_query = ''
category = '4bf58dd8d48988d16e941735'
radius = 20000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&category={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, category, radius, LIMIT)
url

results = requests.get(url).json()
results

 .... OK!


{'meta': {'code': 200, 'requestId': '5cefdd1fdd57970e232fca9b'},
 'response': {'confident': False,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/bridge_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1df941735',
      'name': 'Bridge',
      'pluralName': 'Bridges',
      'primary': True,
      'shortName': 'Bridge'}],
    'hasPerk': False,
    'id': '52642bf111d2f8f5d1cf5195',
    'location': {'cc': 'US',
     'city': 'Falling Waters',
     'country': 'United States',
     'distance': 304,
     'formattedAddress': ['Falling Waters, WV', 'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.560115478506276,
       'lng': -77.88746313040905}],
     'lat': 39.560115478506276,
     'lng': -77.88746313040905,
     'state': 'WV'},
    'name': 'Falling Waters Bridge & Waterfall',
    'referralId': 'v-1559223583'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/d

In [24]:
# Transforming the data from json to create dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,52642bf111d2f8f5d1cf5195,NaN,US,Falling Waters,United States,NaN,304,"[Falling Waters, WV, United States]","[{'label': 'display', 'lng': -77.8874631304090...",39.560115,-77.887463,NaN,WV,Falling Waters Bridge & Waterfall,v-1559223583,NaN
1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,50168de2e4b0f8ec79790a34,NaN,US,Falling Waters,United States,NaN,739,"[Falling Waters, WV 25419, United States]","[{'label': 'display', 'lng': -77.8985464950265...",39.562217,-77.898546,25419,WV,west virgina welcome Center,v-1559223583,NaN
2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,50642a29e4b05b4017ac0bc4,NaN,US,NaN,United States,NaN,1075,"[West Virginia, United States]","[{'label': 'display', 'lng': -77.8892099791917...",39.568842,-77.889210,NaN,West Virginia,By God West Virginia Traffic,v-1559223583,NaN
3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,4bc64452bf29c9b6f18df92a,Williamsport Pike,US,Falling Waters,United States,NaN,97,"[Williamsport Pike, Falling Waters, WV, United...","[{'label': 'display', 'lng': -77.891377, 'lat'...",39.558496,-77.891377,NaN,WV,Little Falls Presbyterian Church,v-1559223583,NaN
4,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,False,598a7e70fd16bb43b17dec65,5680 Hammonds Mill Rd,US,Martinsburg,United States,NaN,2474,"[5680 Hammonds Mill Rd, Martinsburg, WV 25404,...","[{'label': 'display', 'lng': -77.910249, 'lat'...",39.542837,-77.910249,25404,WV,Walmart Vision & Glasses,v-1559223583,NaN


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Falling Waters Bridge & Waterfall,Bridge,NaN,US,Falling Waters,United States,NaN,304,"[Falling Waters, WV, United States]","[{'label': 'display', 'lng': -77.8874631304090...",39.560115,-77.887463,NaN,WV,52642bf111d2f8f5d1cf5195
1,west virgina welcome Center,General Travel,NaN,US,Falling Waters,United States,NaN,739,"[Falling Waters, WV 25419, United States]","[{'label': 'display', 'lng': -77.8985464950265...",39.562217,-77.898546,25419,WV,50168de2e4b0f8ec79790a34
2,By God West Virginia Traffic,Scenic Lookout,NaN,US,NaN,United States,NaN,1075,"[West Virginia, United States]","[{'label': 'display', 'lng': -77.8892099791917...",39.568842,-77.889210,NaN,West Virginia,50642a29e4b05b4017ac0bc4
3,Little Falls Presbyterian Church,Church,Williamsport Pike,US,Falling Waters,United States,NaN,97,"[Williamsport Pike, Falling Waters, WV, United...","[{'label': 'display', 'lng': -77.891377, 'lat'...",39.558496,-77.891377,NaN,WV,4bc64452bf29c9b6f18df92a
4,Walmart Vision & Glasses,Optical Shop,5680 Hammonds Mill Rd,US,Martinsburg,United States,NaN,2474,"[5680 Hammonds Mill Rd, Martinsburg, WV 25404,...","[{'label': 'display', 'lng': -77.910249, 'lat'...",39.542837,-77.910249,25404,WV,598a7e70fd16bb43b17dec65
5,Walmart Grocery Pickup,Grocery Store,5680 Hammonds Mill Rd,US,Martinsburg,United States,NaN,2210,"[5680 Hammonds Mill Rd, Martinsburg, WV 25404,...","[{'label': 'display', 'lng': -77.9087018791946...",39.544964,-77.908702,25404,WV,5c4b7f686dcf04002c4d160c
6,The Potomac Fish & Game Club,Other Great Outdoors,14241 Falling Waters Rd,US,Williamsport,United States,Along the C&O Canal,1416,[14241 Falling Waters Rd (Along the C&O Canal)...,"[{'label': 'display', 'lng': -77.8765479994236...",39.552884,-77.876548,21795,MD,4bae9493f964a520ffc23be3
7,Walmart Photo Center,Photography Lab,5680 Hammonds Mill Rd,US,Martinsburg,United States,NaN,2474,"[5680 Hammonds Mill Rd, Martinsburg, WV 25404,...","[{'label': 'display', 'lng': -77.910249, 'lat'...",39.542837,-77.910249,25404,WV,598983759ec3995203a97244
8,Coinstar,Bank,5680 Hammonds Mill Rd,US,Martinsburg,United States,NaN,2448,"[5680 Hammonds Mill Rd, Martinsburg, WV 25404,...","[{'label': 'display', 'lng': -77.9099, 'lat': ...",39.542900,-77.909900,25404,WV,5058992e5d24dabaa9bc7b9c
9,I-81 Exit 20,Intersection,Interstate 81,US,Martinsburg,United States,NaN,2303,"[Interstate 81, Martinsburg, WV 25419, United ...","[{'label': 'display', 'lng': -77.9113054275512...",39.545882,-77.911305,25419,WV,4c3a1af5ae2da593e1ac03c6


In [39]:
dataframe_filtered[['name','categories', 'distance','lat','lng']]


,name,categories,distance,lat,lng
0,Falling Waters Bridge & Waterfall,Bridge,304,39.560115,-77.887463
1,west virgina welcome Center,General Travel,739,39.562217,-77.898546
2,By God West Virginia Traffic,Scenic Lookout,1075,39.568842,-77.889210
3,Little Falls Presbyterian Church,Church,97,39.558496,-77.891377
4,Walmart Vision & Glasses,Optical Shop,2474,39.542837,-77.910249
5,Walmart Grocery Pickup,Grocery Store,2210,39.544964,-77.908702
6,The Potomac Fish & Game Club,Other Great Outdoors,1416,39.552884,-77.876548
7,Walmart Photo Center,Photography Lab,2474,39.542837,-77.910249
8,Coinstar,Bank,2448,39.542900,-77.909900
9,I-81 Exit 20,Intersection,2303,39.545882,-77.911305


In [41]:
# Map all the business locations in the 10-miles radius centered around Falling Waters, WV.
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Falling Waters, WV',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map